In [2]:
from scipy.signal import find_peaks, peak_prominences, peak_widths
import numpy as np
import matplotlib.pyplot as plt

def peak_det_102(signal=[], fps=30, win_size=1.5):

    peaks_101 = []
    proms = []
    widths = []
    s = 0

    while win_size <= len(signal) / fps:
        x = 1.5
        t = 0

        window_signal = signal[int((win_size - x) * fps): int(win_size * fps)]

        peaks, _ = find_peaks(window_signal, height=0)
        s += len(peaks)
        peaks_reverse, _ = find_peaks((window_signal * -1), height=0)

        if len(peaks) == 0:
            win_size += x
            continue

        proms = peak_prominences(window_signal, peaks)[0]
        widths = peak_widths(window_signal, peaks, rel_height = 0.5)[0]

        if len(widths) == 0 or len(proms) == 0:
            win_size += x
            continue

        width_threshold = 0
        prominence_threshold = 0.5 * np.max(proms)

        peaks_2 = []

        for i in range(len(peaks)):
            if proms[i] < prominence_threshold or widths[i] < width_threshold:
                peaks_2.append(peaks[i])

        peaks_set = set(peaks)
        peaks_2_set = set(peaks_2)

        peaks_1_set = peaks_set - peaks_2_set
        peaks_1 = list(peaks_1_set)
        peaks_1.sort()

        for l in range(len(peaks_1)):
            peaks_1[l] += int((win_size - x) * fps)

        for l1 in range(len(peaks_reverse)):
            peaks_reverse[l1] += int((win_size - x) * fps)

        i = 0

        while i < len(peaks_1) - 1:
            if peaks_1[i + 1] - peaks_1[i] < 9:
                peaks_1.pop(i + 1)
            else:
                i += 1

        while t < len(peaks_1) - 1:
            start = peaks_1[t]
            end = peaks_1[t + 1]

            if any(start < value < end for value in peaks_reverse):
                t += 1
            else:
                peaks_1.pop(t + 1)

        win_size += x
        peaks_101.extend(peaks_1)

        u = 0

        while u < len(peaks_101) - 1:
            if peaks_101[u + 1] - peaks_101[u] < 9:
                peaks_101.pop(u + 1)
            else:
                u += 1

        while u < len(peaks_101) - 1:
            if peaks_101[u + 1] - peaks_101[u] > 45:
                peaks_101.pop(u)
            else:
                u += 1

    return peaks_101, proms, widths

In [4]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks, peak_prominences, peak_widths
import matplotlib.pyplot as plt

csv_file_path = '/content/Akhil_Pre_1.csv'
df = pd.read_csv(csv_file_path)

bpms = []
proms_1 = []
widths_1 = []

fps = 30
win_size = 1.5

zero_count = []

for e in range(df.shape[0]):
  y = df.iloc[e, :]
  zero_count.append(list(y).count(0))

medi = np.median(zero_count)

for e in range(df.shape[0]):
  y = df.iloc[e, :]

  if(list(y).count(0) <= medi):
    t, p, w = peak_det_102(y, fps, win_size)
    proms_1.extend(p)
    widths_1.extend(w)

    diffs = []

    for x in range(len(t) - 1):
      t[x] = int(t[x])
      t[x+1] = int(t[x+1])
      diff = t[x+1] - t[x]
      diffs.append(diff)

    if diffs:
        avg_peak_dist = np.mean(diffs)
        bpm = round(1800 / avg_peak_dist)

    else:
        pass

    bpms.append(bpm)

  else:
    pass

  df2 = pd.DataFrame({"BPM": bpms})
  df2.to_csv("BPM.csv", index = False)

  plt.figure(figsize = (30, 20))
  plt.plot(y)
  plt.title(f"Patch_{e + 1}")
  plt.xticks(rotation = 90)
  plt.plot([x for x in t], y[t], 'x', markersize = 20)
  plt.show()